In [0]:


CREATE STREAMING TABLE yellow_taxi_bronze
COMMENT "Bronze raw NYC taxi data"
TBLPROPERTIES (
  "quality" = "bronze"
)
AS
SELECT
  VendorID,
  CAST(tpep_pickup_datetime AS TIMESTAMP)  AS pickup_datetime,
  CAST(tpep_dropoff_datetime AS TIMESTAMP) AS dropoff_datetime,
  passenger_count,
  trip_distance,
  RatecodeID,
  store_and_fwd_flag,
  PULocationID,
  DOLocationID,
  payment_type,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  improvement_surcharge,
  total_amount,
  congestion_surcharge,
  current_timestamp() AS processing_time,
  _metadata.file_name AS source_file
FROM STREAM(
  read_files(
    '${source}',
    format => 'parquet',
    inferSchema => true
));



In [0]:


CREATE MATERIALIZED VIEW yellow_taxi_silver
COMMENT "Silver cleaned and enriched taxi trips"
TBLPROPERTIES (
  "quality" = "silver"
)
AS
SELECT
    VendorID,

    -- Timestamps
    pickup_datetime,
    dropoff_datetime,
    DATE(pickup_datetime) AS pickup_date,
    HOUR(pickup_datetime) AS pickup_hour,

    -- Trip metrics
    passenger_count,
    trip_distance,

    -- Derived metrics
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 60
        AS trip_duration_minutes,

    CASE
        WHEN trip_distance > 0 THEN
            trip_distance /
            ((unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 3600)
        ELSE NULL
    END AS avg_speed_kmh,

    -- Locations
    PULocationID,
    DOLocationID,

    -- Fare details
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    congestion_surcharge,
    total_amount,

    -- Normalized flags
    CASE
        WHEN store_and_fwd_flag = 'Y' THEN true
        WHEN store_and_fwd_flag = 'N' THEN false
        ELSE null
    END AS store_and_fwd_flag,

    payment_type,

    -- Metadata
    processing_time,
    source_file

FROM yellow_taxi_bronze

-- ✅ Core data quality filters
WHERE
    pickup_datetime IS NOT NULL
    AND dropoff_datetime IS NOT NULL
    AND dropoff_datetime > pickup_datetime
    AND trip_distance > 0
    AND total_amount >= 0;


In [0]:


CREATE MATERIALIZED VIEW yellow_taxi_by_pickup
COMMENT "Daily pickups by date"
TBLPROPERTIES (
  "quality" = "gold"
)
AS
SELECT
    pickup_date,
    COUNT(*) AS total_daily_pickups
FROM yellow_taxi_silver
GROUP BY pickup_date;
